# Census Data for San Antonio Districts

In [1]:
import pandas as pd
import numpy as np
import os
import re
import math

## District Data

This sheet tells which district each census tract aligns with and what percentage of the district is included. District 0 are census tracts outside of San Antonio.

In [2]:
dist = pd.read_excel('CMO_CDTractsPrecincts_220406.xlsx')
dist = dist[['Council District',
             'Tract 2020 ID\n(Use with 2020 Decennial Census)',
             'Percentage of Tract 2020 in Council District']]
dist.columns = ['council','tract','percent']
dist = dist.iloc[1:].reset_index(drop=True)
dist.tract = dist.tract.astype(str)
dist.council = np.where(dist.council == 'Outside CoSA', 0, dist.council)
dist.head()

,council,tract,percent
0,1,110100,0.999534
1,1,110300,0.499153
2,1,110500,0.000150
3,1,110600,0.002611
4,1,110700,0.953113


## Census Data

This is all the data pulled from the census. 

In [3]:
filename = 'ACSDP5Y2021.DP04-Data.csv'
filename2 = 'ACSDP5Y2021.DP04-Column-Metadata.csv'

In [4]:
dataall = pd.read_csv(filename)
dataall.columns = dataall.columns.str.lower()

dataall = dataall.iloc[1:].reset_index(drop=True)
dataall = dataall.replace('(X)', np.nan).replace('-',np.nan).replace('\*+',np.nan,regex=True)
# dataall = dataall.dropna(axis=1, how='all')

dataall.geo_id = dataall.geo_id.str[-6:]
dataall.geo_id = dataall.geo_id.astype(str)
dataall = dataall.drop(columns='name')

to_fix = []
for col in dataall.columns:
    bad_num = dataall[col].astype(str).str.contains('+',regex=False).sum()
    if bad_num > 0:
        to_fix.append(col)
for col in to_fix:
    dataall[col] = dataall[col].astype(str)\
    .str.replace('+','',regex=False)\
    .str.replace(',','').astype(float)

dataall.head()

,geo_id,dp04_0001e,dp04_0001ea,dp04_0001m,dp04_0001ma,dp04_0002e,dp04_0002m,dp04_0002ma,dp04_0002ea,dp04_0003e,...,dp04_0141pea,dp04_0142pe,dp04_0142pm,dp04_0142pma,dp04_0142pea,dp04_0143pe,dp04_0143pm,dp04_0143pma,dp04_0143pea,unnamed: 1146
0,110100,2958,NaN,272,NaN,1895,286,NaN,NaN,1063,...,NaN,31.1,11.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110300,1888,NaN,265,NaN,1393,251,NaN,NaN,495,...,NaN,40.4,14.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110500,1025,NaN,117,NaN,921,133,NaN,NaN,104,...,NaN,41.4,11.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110600,1637,NaN,255,NaN,1551,269,NaN,NaN,86,...,NaN,51.9,13.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110700,661,NaN,116,NaN,513,104,NaN,NaN,148,...,NaN,43.2,18.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Labels

These are the labels for the census data.

In [5]:
labels = pd.read_csv(filename2)
labels.columns = labels.columns.str.lower().str.replace('\W+','_', regex=True)
labels.column_name = labels.column_name.str.lower()

labels = labels [labels.label.str.startswith('Estimate')]
labels = labels [~labels.label.str.contains('ratio')]

groups = labels.label.str.split('!', expand=True)[2].unique()
groups

array(['HOUSING OCCUPANCY', 'UNITS IN STRUCTURE', 'YEAR STRUCTURE BUILT',
       'ROOMS', 'BEDROOMS', 'HOUSING TENURE',
       'YEAR HOUSEHOLDER MOVED INTO UNIT', 'VEHICLES AVAILABLE',
       'HOUSE HEATING FUEL', 'SELECTED CHARACTERISTICS',
       'OCCUPANTS PER ROOM', 'VALUE', 'MORTGAGE STATUS',
       'SELECTED MONTHLY OWNER COSTS (SMOC)',
       'SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)',
       'GROSS RENT',
       'GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)'],
      dtype=object)

## Functions to combine everything

In [6]:
def cal_counts(label):
    #pull out estimate and moa
    name_est = label + 'e'
    name_moa = label + 'm'

    #find the corresponding title to census label
    string = labels [labels.column_name == name_est].label.iloc[0]
#     label_title = re.findall('!!([\w\s]+)$',string)[0]
    if not string.split('!!')[3:]:
        label_title = string.split('!!')[-1]
    else:
        label_title = '!!'.join(string.split('!!')[3:])

    #isolate just one indicator
    data = dataall[['geo_id', name_est, name_moa]].copy()

    #join districts and indicator together
    df = dist.merge(data, how='inner', right_on='geo_id', left_on='tract')
    df = df.replace(np.nan, 0)

    #get count percentages for each census tract per district
    df['tru_count'] = df[name_est].astype(float) * df.percent
    df['tru_error'] = df[name_moa].astype(float) * df.percent

    #for each council district, sum up counts, and calculate new moe
    council_counts = []
    total_counts = df.tru_count.sum()

    for x in df.council.unique():
        subset = df [df.council == x]

        count = round(subset.tru_count.sum(), 2)
        count_perc = round(count / total_counts, 2)

        error = round((subset.tru_error ** 2).sum()**.5, 2)
        error_perc = round(math.sqrt(sum((subset.tru_error / 1.645)**2)) / count, 2)

        council_counts.append([x, count, count_perc, error, error_perc])

    #convert to dataframe and format
    dff = pd.DataFrame(council_counts).sort_values(0)
    dff.columns = [label_title, 'counts', 'count_perc', 'moe', 'moe_perc']
    dff = dff.set_index(label_title, drop=True)
#     print(dff)

    return dff

In [7]:
def cal_district_numbers(labels_check):
    district_totals = pd.DataFrame(np.arange(11))
    totals = []

    for label in labels_check:
        dff = cal_counts(label)
        name = dff.index.name
        dff.columns = [name + ' counts', name + ' counts percent', name + ' moe', name + ' moa percent']
        dff = dff.reset_index(drop=True)

        district_totals = pd.concat([district_totals,dff],axis=1)

    district_totals = district_totals.drop(columns=0)
    return district_totals.T

## Now calculate! 

### Housing units

In [8]:
grouping = groups[0]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-13]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

HOUSING OCCUPANCY

Estimate!!HOUSING OCCUPANCY!!Total housing units
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Homeowner vacancy rate
Estimate!!HOUSING OCCUPANCY!!Total housing units!!Rental vacancy rate


In [9]:
final1 = cal_district_numbers(labels_check)
final1

,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,188661.64,57123.02,57254.27,53818.39,43705.62,45540.96,61319.53,61352.55,77166.25,69283.60,62837.16
Total housing units counts percent,0.24,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Total housing units moe,2501.64,1317.08,1360.16,1189.09,939.71,1163.11,1410.20,1362.88,1474.12,1445.48,1184.30
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01
Occupied housing units counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Occupied housing units counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Occupied housing units moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Vacant housing units counts,11331.85,7348.71,6032.14,4718.24,2821.09,4579.05,3356.92,4875.67,7410.68,4550.23,3914.41
Vacant housing units counts percent,0.19,0.12,0.10,0.08,0.05,0.08,0.06,0.08,0.12,0.07,0.06


### Units in structure

In [10]:
grouping = groups[1]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

UNITS IN STRUCTURE

Estimate!!UNITS IN STRUCTURE!!Total housing units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!1-unit, detached
Estimate!!UNITS IN STRUCTURE!!Total housing units!!1-unit, attached
Estimate!!UNITS IN STRUCTURE!!Total housing units!!2 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!3 or 4 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!5 to 9 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!10 to 19 units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!20 or more units
Estimate!!UNITS IN STRUCTURE!!Total housing units!!Mobile home
Estimate!!UNITS IN STRUCTURE!!Total housing units!!Boat, RV, van, etc.


In [11]:
final2 = cal_district_numbers(labels_check)    
final2

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in double_scalars


,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,188661.64,57123.02,57254.27,53818.39,43705.62,45540.96,61319.53,61352.55,77166.25,69283.60,62837.16
Total housing units counts percent,0.24,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Total housing units moe,2501.64,1317.08,1360.16,1189.09,939.71,1163.11,1410.20,1362.88,1474.12,1445.48,1184.30
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01
"1-unit, detached counts",147676.57,31288.53,37142.12,37583.50,33618.75,34393.06,41071.34,34854.69,29592.71,38040.00,41369.71
"1-unit, detached counts percent",0.29,0.06,0.07,0.07,0.07,0.07,0.08,0.07,0.06,0.08,0.08
"1-unit, detached moe",2404.12,1072.21,1072.98,1094.90,903.03,1057.59,1312.01,1103.70,958.50,1003.85,1027.02
"1-unit, detached moa percent",0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
"1-unit, attached counts",3823.98,1294.55,1482.10,786.77,813.48,908.83,1092.60,2582.26,1873.25,1542.99,1662.19
"1-unit, attached counts percent",0.21,0.07,0.08,0.04,0.05,0.05,0.06,0.14,0.10,0.09,0.09


### Year Built

In [12]:
grouping = groups[2]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

YEAR STRUCTURE BUILT

Estimate!!YEAR STRUCTURE BUILT!!Total housing units
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2020 or later
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2010 to 2019
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 2000 to 2009
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1990 to 1999
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1980 to 1989
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1970 to 1979
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1960 to 1969
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1950 to 1959
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1940 to 1949
Estimate!!YEAR STRUCTURE BUILT!!Total housing units!!Built 1939 or earlier


In [13]:
final3 = cal_district_numbers(labels_check)
final3

,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,188661.64,57123.02,57254.27,53818.39,43705.62,45540.96,61319.53,61352.55,77166.25,69283.60,62837.16
Total housing units counts percent,0.24,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Total housing units moe,2501.64,1317.08,1360.16,1189.09,939.71,1163.11,1410.20,1362.88,1474.12,1445.48,1184.30
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01
Built 2020 or later counts,1995.13,23.06,383.32,97.40,120.84,35.40,170.64,149.37,77.78,135.74,88.31
Built 2020 or later counts percent,0.61,0.01,0.12,0.03,0.04,0.01,0.05,0.05,0.02,0.04,0.03
Built 2020 or later moe,396.65,87.30,173.70,111.17,105.52,86.49,159.35,124.80,105.77,133.81,101.21
Built 2020 or later moa percent,0.12,2.30,0.28,0.69,0.53,1.49,0.57,0.51,0.83,0.60,0.70
Built 2010 to 2019 counts,52936.95,3938.05,6617.04,5121.28,4058.69,1753.59,8611.38,3617.21,14767.07,7879.29,5492.46
Built 2010 to 2019 counts percent,0.46,0.03,0.06,0.04,0.04,0.02,0.08,0.03,0.13,0.07,0.05


### Rooms

In [14]:
grouping = groups[3]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
current_labels = current_labels.iloc[:-7]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

ROOMS

Estimate!!ROOMS!!Total housing units
Estimate!!ROOMS!!Total housing units!!1 room
Estimate!!ROOMS!!Total housing units!!2 rooms
Estimate!!ROOMS!!Total housing units!!3 rooms
Estimate!!ROOMS!!Total housing units!!4 rooms
Estimate!!ROOMS!!Total housing units!!5 rooms
Estimate!!ROOMS!!Total housing units!!6 rooms
Estimate!!ROOMS!!Total housing units!!7 rooms
Estimate!!ROOMS!!Total housing units!!8 rooms
Estimate!!ROOMS!!Total housing units!!9 rooms or more
Estimate!!ROOMS!!Total housing units!!Median rooms


In [15]:
final4 = cal_district_numbers(labels_check)
final4

,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,188661.64,57123.02,57254.27,53818.39,43705.62,45540.96,61319.53,61352.55,77166.25,69283.60,62837.16
Total housing units counts percent,0.24,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Total housing units moe,2501.64,1317.08,1360.16,1189.09,939.71,1163.11,1410.20,1362.88,1474.12,1445.48,1184.30
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01
1 room counts,2636.95,3387.01,1534.98,1686.14,800.65,1435.98,1528.63,2015.08,3676.73,1649.37,1616.48
1 room counts percent,0.12,0.15,0.07,0.08,0.04,0.07,0.07,0.09,0.17,0.08,0.07
1 room moe,465.47,509.71,304.17,325.98,253.84,289.52,352.19,353.45,536.45,465.64,384.33
1 room moa percent,0.11,0.09,0.12,0.12,0.19,0.12,0.14,0.11,0.09,0.17,0.14
2 rooms counts,4203.58,2928.19,2565.53,2367.18,1102.82,1248.58,2458.20,2661.25,6580.10,2923.77,2349.78
2 rooms counts percent,0.13,0.09,0.08,0.08,0.04,0.04,0.08,0.08,0.21,0.09,0.07


### Bedrooms

In [16]:
grouping = groups[4]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

BEDROOMS

Estimate!!BEDROOMS!!Total housing units
Estimate!!BEDROOMS!!Total housing units!!No bedroom
Estimate!!BEDROOMS!!Total housing units!!1 bedroom
Estimate!!BEDROOMS!!Total housing units!!2 bedrooms
Estimate!!BEDROOMS!!Total housing units!!3 bedrooms
Estimate!!BEDROOMS!!Total housing units!!4 bedrooms
Estimate!!BEDROOMS!!Total housing units!!5 or more bedrooms


In [17]:
final5 = cal_district_numbers(labels_check)
final5

,0,1,2,3,4,5,6,7,8,9,10
Total housing units counts,188661.64,57123.02,57254.27,53818.39,43705.62,45540.96,61319.53,61352.55,77166.25,69283.60,62837.16
Total housing units counts percent,0.24,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Total housing units moe,2501.64,1317.08,1360.16,1189.09,939.71,1163.11,1410.20,1362.88,1474.12,1445.48,1184.30
Total housing units moa percent,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01
No bedroom counts,2916.44,3671.44,1598.19,1745.86,909.37,1485.16,1598.26,2188.76,3757.39,1776.66,1679.48
No bedroom counts percent,0.13,0.16,0.07,0.07,0.04,0.06,0.07,0.09,0.16,0.08,0.07
No bedroom moe,486.49,518.43,305.27,327.22,259.66,292.35,356.79,367.94,537.93,474.77,387.45
No bedroom moa percent,0.10,0.09,0.12,0.11,0.17,0.12,0.14,0.10,0.09,0.16,0.14
1 bedroom counts,11274.86,13418.91,8716.29,7215.44,3511.28,6751.31,8120.70,10500.16,19920.22,14169.63,8096.21
1 bedroom counts percent,0.10,0.12,0.08,0.06,0.03,0.06,0.07,0.09,0.18,0.13,0.07


### Tenure

In [18]:
grouping = groups[5]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-2]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final6 = cal_district_numbers(labels_check)
final6

HOUSING TENURE

Estimate!!HOUSING TENURE!!Occupied housing units
Estimate!!HOUSING TENURE!!Occupied housing units!!Owner-occupied
Estimate!!HOUSING TENURE!!Occupied housing units!!Renter-occupied
Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of owner-occupied unit
Estimate!!HOUSING TENURE!!Occupied housing units!!Average household size of renter-occupied unit


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Occupied housing units counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Occupied housing units moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Owner-occupied counts,134220.34,22401.87,25273.12,29062.45,25749.50,22986.08,33563.21,29316.87,26933.87,35075.54,36302.14
Owner-occupied counts percent,0.32,0.05,0.06,0.07,0.06,0.05,0.08,0.07,0.06,0.08,0.09
Owner-occupied moe,2410.04,981.96,1018.04,1065.10,916.31,951.43,1277.99,1116.46,955.98,919.42,1105.32
Owner-occupied moa percent,0.01,0.03,0.02,0.02,0.02,0.03,0.02,0.02,0.02,0.02,0.02
Renter-occupied counts,43109.46,27372.44,25949.01,20037.69,15135.03,17975.83,24399.40,27160.01,42821.70,29657.82,22620.61
Renter-occupied counts percent,0.15,0.09,0.09,0.07,0.05,0.06,0.08,0.09,0.14,0.10,0.08


### YEAR HOUSEHOLDER MOVED INTO UNIT

In [19]:
grouping = groups[6]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-2]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final7 = cal_district_numbers(labels_check)
final7

YEAR HOUSEHOLDER MOVED INTO UNIT

Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 2019 or later
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 2015 to 2018
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 2010 to 2014
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 2000 to 2009
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 1990 to 1999
Estimate!!YEAR HOUSEHOLDER MOVED INTO UNIT!!Occupied housing units!!Moved in 1989 and earlier


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Occupied housing units counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Occupied housing units moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Moved in 2019 or later counts,21928.18,6841.88,7262.37,5138.09,4007.53,3661.95,8037.04,8189.72,15403.10,9907.66,7354.48
Moved in 2019 or later counts percent,0.22,0.07,0.07,0.05,0.04,0.04,0.08,0.08,0.16,0.10,0.08
Moved in 2019 or later moe,1441.26,710.62,786.76,678.61,498.39,587.58,912.88,841.57,1192.67,1084.66,815.01
Moved in 2019 or later moa percent,0.04,0.06,0.07,0.08,0.08,0.10,0.07,0.06,0.05,0.07,0.07
Moved in 2015 to 2018 counts,62343.66,17370.72,17814.34,14138.69,10710.59,9683.09,17911.14,18072.66,29916.63,25408.03,20766.44
Moved in 2015 to 2018 counts percent,0.26,0.07,0.07,0.06,0.04,0.04,0.07,0.07,0.12,0.10,0.09


### VEHICLES AVAILABLE

In [20]:
grouping = groups[7]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-2]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final8 = cal_district_numbers(labels_check)
final8

VEHICLES AVAILABLE

Estimate!!VEHICLES AVAILABLE!!Occupied housing units
Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!No vehicles available
Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!1 vehicle available
Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!2 vehicles available
Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Occupied housing units counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Occupied housing units moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
No vehicles available counts,4517.17,5855.25,4892.91,5917.46,2253.83,6237.30,2403.54,5093.38,3580.78,2761.20,3013.19
No vehicles available counts percent,0.10,0.13,0.11,0.13,0.05,0.13,0.05,0.11,0.08,0.06,0.06
No vehicles available moe,502.91,622.59,497.52,706.02,356.98,682.06,478.26,587.12,554.79,473.87,529.72
No vehicles available moa percent,0.07,0.06,0.06,0.07,0.10,0.07,0.12,0.07,0.09,0.10,0.11
1 vehicle available counts,46086.33,20493.35,20095.70,16576.77,12707.69,15056.02,20036.70,22669.55,30255.14,23471.57,21011.18
1 vehicle available counts percent,0.19,0.08,0.08,0.07,0.05,0.06,0.08,0.09,0.12,0.09,0.08


### HOUSE HEATING FUEL

In [21]:
grouping = groups[8]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final9 = cal_district_numbers(labels_check)
final9

HOUSE HEATING FUEL

Estimate!!HOUSE HEATING FUEL!!Occupied housing units
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Utility gas
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Bottled, tank, or LP gas
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Electricity
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Fuel oil, kerosene, etc.
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Coal or coke
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Wood
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Solar energy
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!Other fuel
Estimate!!HOUSE HEATING FUEL!!Occupied housing units!!No fuel used


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in double_scalars


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Occupied housing units counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Occupied housing units moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Utility gas counts,47320.11,16279.75,15843.94,18295.25,11897.15,17349.78,15004.37,17993.77,15677.23,17905.04,17653.58
Utility gas counts percent,0.22,0.08,0.08,0.09,0.06,0.08,0.07,0.09,0.07,0.08,0.08
Utility gas moe,1520.70,793.79,916.55,971.92,784.25,925.06,997.44,966.46,853.54,832.37,925.48
Utility gas moa percent,0.02,0.03,0.04,0.03,0.04,0.03,0.04,0.03,0.03,0.03,0.03
"Bottled, tank, or LP gas counts",3495.53,232.85,398.83,447.62,372.17,224.97,260.23,216.50,444.38,776.66,343.26
"Bottled, tank, or LP gas counts percent",0.48,0.03,0.06,0.06,0.05,0.03,0.04,0.03,0.06,0.11,0.05


### SELECTED CHARACTERISTICS

In [22]:
grouping = groups[9]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final10 = cal_district_numbers(labels_check)
final10

SELECTED CHARACTERISTICS

Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units
Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete plumbing facilities
Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!Lacking complete kitchen facilities
Estimate!!SELECTED CHARACTERISTICS!!Occupied housing units!!No telephone service available


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Occupied housing units counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Occupied housing units moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
Lacking complete plumbing facilities counts,919.05,252.64,439.02,300.38,232.51,270.55,197.94,352.99,136.10,69.18,148.65
Lacking complete plumbing facilities counts percent,0.28,0.08,0.13,0.09,0.07,0.08,0.06,0.11,0.04,0.02,0.04
Lacking complete plumbing facilities moe,384.10,155.77,183.30,153.66,132.10,118.19,128.24,177.05,160.35,105.90,124.83
Lacking complete plumbing facilities moa percent,0.25,0.37,0.25,0.31,0.35,0.27,0.39,0.30,0.72,0.93,0.51
Lacking complete kitchen facilities counts,635.23,538.29,336.82,445.42,246.25,371.69,220.42,431.53,539.89,205.32,573.15
Lacking complete kitchen facilities counts percent,0.14,0.12,0.07,0.10,0.05,0.08,0.05,0.09,0.12,0.05,0.13


### OCCUPANTS PER ROOM

In [23]:
grouping = groups[10]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final11 = cal_district_numbers(labels_check)
final11

OCCUPANTS PER ROOM

Estimate!!OCCUPANTS PER ROOM!!Occupied housing units
Estimate!!OCCUPANTS PER ROOM!!Occupied housing units!!1.00 or less
Estimate!!OCCUPANTS PER ROOM!!Occupied housing units!!1.01 to 1.50
Estimate!!OCCUPANTS PER ROOM!!Occupied housing units!!1.51 or more


,0,1,2,3,4,5,6,7,8,9,10
Occupied housing units counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Occupied housing units counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Occupied housing units moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Occupied housing units moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
1.00 or less counts,171674.33,47055.76,48804.58,45934.71,37656.38,37220.18,55364.03,54124.13,67048.90,62788.56,57189.44
1.00 or less counts percent,0.25,0.07,0.07,0.07,0.05,0.05,0.08,0.08,0.10,0.09,0.08
1.00 or less moe,2543.67,1373.70,1395.16,1250.50,1014.68,1160.41,1483.06,1391.59,1538.52,1397.98,1212.99
1.00 or less moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.01
1.01 to 1.50 counts,3543.89,1629.11,1618.02,2451.12,2472.47,2642.09,1716.12,1538.26,1610.44,1372.92,1098.57
1.01 to 1.50 counts percent,0.16,0.08,0.07,0.11,0.11,0.12,0.08,0.07,0.07,0.06,0.05


### VALUE

In [24]:
grouping = groups[11]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final12 = cal_district_numbers(labels_check)
final12

VALUE

Estimate!!VALUE!!Owner-occupied units
Estimate!!VALUE!!Owner-occupied units!!Less than $50,000
Estimate!!VALUE!!Owner-occupied units!!$50,000 to $99,999
Estimate!!VALUE!!Owner-occupied units!!$100,000 to $149,999
Estimate!!VALUE!!Owner-occupied units!!$150,000 to $199,999
Estimate!!VALUE!!Owner-occupied units!!$200,000 to $299,999
Estimate!!VALUE!!Owner-occupied units!!$300,000 to $499,999
Estimate!!VALUE!!Owner-occupied units!!$500,000 to $999,999
Estimate!!VALUE!!Owner-occupied units!!$1,000,000 or more
Estimate!!VALUE!!Owner-occupied units!!Median (dollars)


,0,1,2,3,4,5,6,7,8,9,10
Owner-occupied units counts,134220.34,22401.87,25273.12,29062.45,25749.50,22986.08,33563.21,29316.87,26933.87,35075.54,36302.14
Owner-occupied units counts percent,0.32,0.05,0.06,0.07,0.06,0.05,0.08,0.07,0.06,0.08,0.09
Owner-occupied units moe,2410.04,981.96,1018.04,1065.10,916.31,951.43,1277.99,1116.46,955.98,919.42,1105.32
Owner-occupied units moa percent,0.01,0.03,0.02,0.02,0.02,0.03,0.02,0.02,0.02,0.02,0.02
"Less than $50,000 counts",4735.71,1136.51,2467.91,1971.27,1965.67,3609.87,1527.82,829.43,520.40,517.76,904.67
"Less than $50,000 counts percent",0.23,0.06,0.12,0.10,0.10,0.18,0.08,0.04,0.03,0.03,0.04
"Less than $50,000 moe",491.81,247.61,340.68,296.45,399.19,371.04,285.73,232.08,199.35,184.12,219.40
"Less than $50,000 moa percent",0.06,0.13,0.08,0.09,0.12,0.06,0.11,0.17,0.23,0.22,0.15
"$50,000 to $99,999 counts",8381.65,4137.32,6417.82,11203.36,7727.06,12444.16,3945.12,3431.61,799.12,515.02,1632.76
"$50,000 to $99,999 counts percent",0.14,0.07,0.11,0.18,0.13,0.21,0.07,0.06,0.01,0.01,0.03


### MORTGAGE STATUS

In [25]:
grouping = groups[12]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final13 = cal_district_numbers(labels_check)
final13

MORTGAGE STATUS

Estimate!!MORTGAGE STATUS!!Owner-occupied units
Estimate!!MORTGAGE STATUS!!Owner-occupied units!!Housing units with a mortgage
Estimate!!MORTGAGE STATUS!!Owner-occupied units!!Housing units without a mortgage


,0,1,2,3,4,5,6,7,8,9,10
Owner-occupied units counts,134220.34,22401.87,25273.12,29062.45,25749.50,22986.08,33563.21,29316.87,26933.87,35075.54,36302.14
Owner-occupied units counts percent,0.32,0.05,0.06,0.07,0.06,0.05,0.08,0.07,0.06,0.08,0.09
Owner-occupied units moe,2410.04,981.96,1018.04,1065.10,916.31,951.43,1277.99,1116.46,955.98,919.42,1105.32
Owner-occupied units moa percent,0.01,0.03,0.02,0.02,0.02,0.03,0.02,0.02,0.02,0.02,0.02
Housing units with a mortgage counts,93969.37,11061.82,14120.39,14452.20,14374.47,7533.57,21665.89,17182.25,17675.24,23275.36,23341.43
Housing units with a mortgage counts percent,0.36,0.04,0.05,0.06,0.06,0.03,0.08,0.07,0.07,0.09,0.09
Housing units with a mortgage moe,2296.01,729.22,846.05,837.12,726.88,582.20,1142.23,974.07,860.89,860.51,980.20
Housing units with a mortgage moa percent,0.01,0.04,0.04,0.04,0.03,0.05,0.03,0.03,0.03,0.02,0.03
Housing units without a mortgage counts,40250.97,11340.05,11152.73,14610.25,11375.04,15452.51,11897.32,12134.62,9258.63,11800.19,12960.71
Housing units without a mortgage counts percent,0.25,0.07,0.07,0.09,0.07,0.10,0.07,0.07,0.06,0.07,0.08


### SELECTED MONTHLY OWNER COSTS

In [26]:
grouping = groups[13]
grouping = grouping[:-7]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final14 = cal_district_numbers(labels_check)
final14

SELECTED MONTHLY OWNER COSTS

Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!Less than $500
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$500 to $999
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$1,000 to $1,499
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$1,500 to $1,999
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$2,000 to $2,499
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$2,500 to $2,999
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!$3,000 or more
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units with a mortgage!!Median (dollars)
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units without a mortgage
Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!!Housing units 

,0,1,2,3,4,5,6,7,8,9,10
Housing units with a mortgage counts,93969.37,11061.82,14120.39,14452.20,14374.47,7533.57,21665.89,17182.25,17675.24,23275.36,23341.43
Housing units with a mortgage counts percent,0.36,0.04,0.05,0.06,0.06,0.03,0.08,0.07,0.07,0.09,0.09
Housing units with a mortgage moe,2296.01,729.22,846.05,837.12,726.88,582.20,1142.23,974.07,860.89,860.51,980.20
Housing units with a mortgage moa percent,0.01,0.04,0.04,0.04,0.03,0.05,0.03,0.03,0.03,0.02,0.03
Less than $500 counts,593.82,143.35,100.93,139.28,159.94,109.44,153.05,190.96,195.11,24.24,187.87
...,...,...,...,...,...,...,...,...,...,...,...
35.0 percent or more moa percent,0.09,0.17,0.12,0.17,0.16,0.13,0.14,0.16,0.20,0.13,0.19
Not computed counts,530.91,159.51,375.20,337.45,146.51,378.62,70.55,474.46,163.29,125.34,185.18
Not computed counts percent,0.18,0.05,0.13,0.11,0.05,0.13,0.02,0.16,0.06,0.04,0.06
Not computed moe,229.29,100.29,181.42,148.46,119.80,137.28,97.75,248.50,120.01,103.76,114.58


### SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME

In [27]:
grouping = groups[14]
grouping = grouping[:-10]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final15 = cal_district_numbers(labels_check)
final15

SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME

Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!Less than 20.0 percent
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!20.0 to 24.9 percent
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!25.0 to 29.9 percent
Estimate!!SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI)!!Housing units with a mortgage (excluding units where SMOCAPI cannot be computed)!!30.0 to 34.9 percent
Estimate!

,0,1,2,3,4,5,6,7,8,9,10
Housing units with a mortgage (excluding units where SMOCAPI cannot be computed) counts,93521.41,11034.73,14007.26,14404.53,14248.60,7495.31,21603.90,16956.24,17535.76,23094.26,23239.00
Housing units with a mortgage (excluding units where SMOCAPI cannot be computed) counts percent,0.36,0.04,0.05,0.06,0.06,0.03,0.08,0.07,0.07,0.09,0.09
Housing units with a mortgage (excluding units where SMOCAPI cannot be computed) moe,2294.11,728.79,843.84,837.87,725.99,581.86,1139.04,969.35,851.68,858.89,981.02
Housing units with a mortgage (excluding units where SMOCAPI cannot be computed) moa percent,0.01,0.04,0.04,0.04,0.03,0.05,0.03,0.03,0.03,0.02,0.03
Less than 20.0 percent counts,42943.52,4750.47,6134.30,6110.99,5983.45,3217.08,10599.90,8185.52,8591.33,10419.99,9803.44
...,...,...,...,...,...,...,...,...,...,...,...
35.0 percent or more moa percent,0.09,0.17,0.12,0.17,0.16,0.13,0.14,0.16,0.20,0.13,0.19
Not computed counts,530.91,159.51,375.20,337.45,146.51,378.62,70.55,474.46,163.29,125.34,185.18
Not computed counts percent,0.18,0.05,0.13,0.11,0.05,0.13,0.02,0.16,0.06,0.04,0.06
Not computed moe,229.29,100.29,181.42,148.46,119.80,137.28,97.75,248.50,120.01,103.76,114.58


### GROSS RENT

In [28]:
grouping = groups[15]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[:-8]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final16 = cal_district_numbers(labels_check)
final16

GROSS RENT

Estimate!!GROSS RENT!!Occupied units paying rent
Estimate!!GROSS RENT!!Occupied units paying rent!!Less than $500
Estimate!!GROSS RENT!!Occupied units paying rent!!$500 to $999
Estimate!!GROSS RENT!!Occupied units paying rent!!$1,000 to $1,499
Estimate!!GROSS RENT!!Occupied units paying rent!!$1,500 to $1,999
Estimate!!GROSS RENT!!Occupied units paying rent!!$2,000 to $2,499
Estimate!!GROSS RENT!!Occupied units paying rent!!$2,500 to $2,999
Estimate!!GROSS RENT!!Occupied units paying rent!!$3,000 or more
Estimate!!GROSS RENT!!Occupied units paying rent!!Median (dollars)
Estimate!!GROSS RENT!!Occupied units paying rent!!No rent paid


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in double_scalars


,0,1,2,3,4,5,6,7,8,9,10
Occupied units paying rent counts,41389.75,26274.10,24516.87,18531.06,14442.77,16334.37,23724.45,26251.79,41982.07,29139.65,21670.11
Occupied units paying rent counts percent,0.15,0.09,0.09,0.07,0.05,0.06,0.08,0.09,0.15,0.10,0.08
Occupied units paying rent moe,1583.49,1132.17,1250.44,1002.71,864.71,918.67,1158.68,1094.00,1432.25,1333.73,959.76
Occupied units paying rent moa percent,0.02,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.02,0.03,0.03
Less than $500 counts,888.74,2263.61,2331.60,2325.51,751.01,3379.64,668.22,2242.88,450.70,170.79,445.32
Less than $500 counts percent,0.06,0.14,0.15,0.15,0.05,0.21,0.04,0.14,0.03,0.01,0.03
Less than $500 moe,241.63,323.18,350.63,389.48,231.52,428.46,221.01,362.42,195.88,113.18,174.09
Less than $500 moa percent,0.17,0.09,0.09,0.10,0.19,0.08,0.20,0.10,0.26,0.40,0.24
$500 to $999 counts,9213.19,10700.82,9286.08,8625.39,6426.76,8222.24,7397.09,9826.65,11097.68,7473.23,5998.88
$500 to $999 counts percent,0.10,0.11,0.10,0.09,0.07,0.09,0.08,0.10,0.12,0.08,0.06


### GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME

In [29]:
grouping = groups[16]
grouping = grouping[:-8]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)
    
final17 = cal_district_numbers(labels_check)
final17

GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME

Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)
Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!Less than 15.0 percent
Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!15.0 to 19.9 percent
Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!20.0 to 24.9 percent
Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!25.0 to 29.9 percent
Estimate!!GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)!!Occupied units paying rent (excluding units where GRAPI cannot be computed)!!30.0 to 3

,0,1,2,3,4,5,6,7,8,9,10
Occupied units paying rent (excluding units where GRAPI cannot be computed) counts,40931.74,25831.94,23557.33,17925.66,14304.84,15819.00,23550.67,25534.66,40734.86,28744.26,21188.04
Occupied units paying rent (excluding units where GRAPI cannot be computed) counts percent,0.15,0.09,0.08,0.06,0.05,0.06,0.08,0.09,0.15,0.10,0.08
Occupied units paying rent (excluding units where GRAPI cannot be computed) moe,1582.03,1130.73,1240.47,990.24,865.02,913.46,1159.83,1086.25,1431.70,1336.44,953.22
Occupied units paying rent (excluding units where GRAPI cannot be computed) moa percent,0.02,0.03,0.03,0.03,0.04,0.04,0.03,0.03,0.02,0.03,0.03
Less than 15.0 percent counts,5509.57,2711.10,2139.50,2110.56,1121.32,1835.63,2389.75,2449.83,4214.29,3099.44,2319.02
Less than 15.0 percent counts percent,0.18,0.09,0.07,0.07,0.04,0.06,0.08,0.08,0.14,0.10,0.08
Less than 15.0 percent moe,738.34,479.80,397.70,394.60,358.13,345.58,492.56,411.36,567.46,562.40,391.91
Less than 15.0 percent moa percent,0.08,0.11,0.11,0.11,0.19,0.11,0.13,0.10,0.08,0.11,0.10
15.0 to 19.9 percent counts,5458.91,3085.32,2685.05,1920.33,1722.04,1594.39,3000.91,3060.66,5883.20,3374.11,2285.08
15.0 to 19.9 percent counts percent,0.16,0.09,0.08,0.06,0.05,0.05,0.09,0.09,0.17,0.10,0.07


In [30]:
with pd.ExcelWriter('sa2020_ccd_housing.xlsx') as writer:
    final1.to_excel(writer, sheet_name=groups[0][:30])
    final2.to_excel(writer, sheet_name=groups[1][:30])
    final3.to_excel(writer, sheet_name=groups[2][:30])
    final4.to_excel(writer, sheet_name=groups[3][:30])
    final5.to_excel(writer, sheet_name=groups[4][:30])
    final6.to_excel(writer, sheet_name=groups[5][:30])
    final7.to_excel(writer, sheet_name=groups[6][:30])
    final8.to_excel(writer, sheet_name=groups[7][:30])
    final9.to_excel(writer, sheet_name=groups[7][:30])
    final10.to_excel(writer, sheet_name=groups[7][:30])
    final11.to_excel(writer, sheet_name=groups[7][:30])        
    final12.to_excel(writer, sheet_name=groups[7][:30])
    final13.to_excel(writer, sheet_name=groups[7][:30])
    final14.to_excel(writer, sheet_name=groups[7][:30])
    final15.to_excel(writer, sheet_name=groups[7][:30])
    final16.to_excel(writer, sheet_name=groups[7][:30])
    final17.to_excel(writer, sheet_name=groups[7][:30])

In [31]:
len(groups)

17